# Find well conformers and calculate energy on the fly
This uses a RDkit as backend to generate the conformer and calculate the energy by Psi4.

Note: if the jupyter notebook crashes. It is highly possible that the `openbabel` and `rdkit` <br>
uses different dependencies during compiling, assuming you are using conda builds.<br>
You can either try to build your own version (solve the problem) or change the import sequence<br>
(can make the latter one malfunctioning when use some of the methods)<br>

## 1. Generate conformers
Necessary packages

In [ ]:
import os
import multiprocessing
from copy import deepcopy
from itertools import combinations, product
from typing import List, Optional, Union

import numpy as np
import psi4
from joblib import Parallel, delayed

from acs.backend.rdk import RDKitMol, RDKitConf
from acs.converter.geom import (xyz_str_to_xyz_dict,
                                xyz_dict_to_xyz_str,
                                xyz_dict_to_xyz_file)
import pybel

%load_ext autoreload
%load_ext autotime
%autoreload 2

In [ ]:
def get_separable_angle_list(samplings: Union[list, tuple],
                             from_angles: Optional[Union[list, tuple]] = None):
    """
    Get a angle list for each input dimension. For each dimension
    The input can be a int, indicating the angles will be evenly sampled;
    Or a list, indicate the angles to be sampled;
    Examples:
    [[120, 240,], 4, 0] => [np.array([120, 240,]), 
                            np.array([0, 90, 180, 270,]), 
                            np.array([0])]
    List of np.arrays are returned for the sake of further calculation
    
    Args:
        samplings (Union[list, tuple]): An array of sampling information.
                  For each element, it can be either list or int.
        from_angles (Union[list, tuple]): An array of initial angles.
                    If not set, angles will begin at zeros.
    
    Returns:
        list: A list of sampled angles sets.
    """
    from_angles = from_angles or len(samplings) * [0.]
    angle_list = []
    for ind, angles in enumerate(samplings):
        # Only provide a number
        # This is the step number of the angles
        if isinstance(angles, (int, float)):
            try:
                step = 360 // angles
            except ZeroDivisionError:
                # Does not change
                angles = from_angles[ind] + np.array([0])
            else:
                angles = from_angles[ind] + \
                         np.array([step * i for i in range(angles)])
        elif isinstance(angles, list):
            angles = from_angles[ind] + np.array(angles)
            
        # Set to 0 - 360 range
        for i in range(angles.shape[0]):
            while angles[i] < 0.:
                angles[i] += 360
            while angles[i] > 360.:
                angles[i] -= 360

        angle_list.append(angles.tolist())            
    return angle_list


def conformers_by_change_torsions(conf: 'RDKitConf',
                                  angle_mesh,
                                  bookkeep: dict,
                                  torsions=None,
                                  on_the_fly_check=False):
    """
    Generate conformers by rotating the angles of the torsions. The result will be saved into
    ``bookkeep``. A on-the-fly check can be applied, which identifies the conformers with colliding
    atoms.
    
    Args:
        conf (RDkitConf): A RDKit Conformer to be used.
        angle_mesh (iterable): An iterable contains the angle_list for conformers to be generated from.
        bookkeep (dict): A dictionary to save the coords.
        torsions (list): A list of four-atom-index lists indicating the torsional modes.
        on_the_fly_filter (bool): Whether to check colliding atoms on the fly.
    """
    if not torsions:
        torsions = conf.GetTorsionalModes()
        for ind, angles in enumerate(angle_mesh):
            conf.SetAllTorsionsDeg(angles)
            bookkeep[ind] = {'angles': angles,
                             'coords': conf.GetPositions().tolist()}
            bookkeep[ind]['colliding_atoms'] = conf.HasCollidingAtoms() \
                if on_the_fly_check == True else None

    else:
        all_torsions = conf.GetTorsionalModes()
        try:
            changing_torsions_index = [all_torsions.index(tor) for tor in torsions]
        except ValueError as e:
            # tor not in all_torsions
            raise
            
        original_angles = conf.GetAllTorsionsDeg()

        for ind, angles in enumerate(angle_mesh):
            for i, angle, tor in zip(range(len(angles)), angles, torsions):
                conf.SetTorsionDeg(tor, angle)
                original_angles[changing_torsions_index[i]] = angle
                
            bookkeep[ind] = {'angles': original_angles,
                             'coords': conf.GetPositions().tolist()}
            bookkeep[ind]['colliding_atoms'] = conf.HasCollidingAtoms() \
                    if on_the_fly_check == True else None

## 1.1 Perceive a molecule from xyz

In [ ]:
######################################
# INPUT
xyz_str="""O       1.85073500   -1.14075800   -0.80808500
O       1.64413700   -0.98958800    0.61920900
C       0.89993200    0.19167500    0.83332300
C       1.57997200    1.46553500    0.32458600
C       0.87059800    2.73251700    0.76632100
O       2.94131500    1.54377500    0.86706900
O       3.78534700    0.75777800    0.22699800
H       2.74479300   -0.76185300   -0.87545700
H       0.78501500    0.22753800    1.92086600
H      -0.09098800    0.11998200    0.36665200
H       1.69085200    1.41865800   -0.75897600
H       0.84097800    2.80358600    1.85617500
H      -0.15579100    2.73441900    0.39232200
H       1.37972400    3.61444100    0.37512000"""
######################################

Pybel (openbabel) is used to perceive the connectivity

In [ ]:
xyz_file = f"{len(xyz_str.splitlines())}\n\n{xyz_str}"
xyz_dict = xyz_str_to_xyz_dict(xyz_str)
pybel_mol = pybel.readstring('xyz', xyz_file)
pybel_mol

## 1.1 Use RDKit to generate conformers

Creat RDKit molecule and RDkit Conformer

In [ ]:
# Generate a RDKit Molecule
rdkitmol = RDKitMol.FromOBMol(pybel_mol.OBMol)

# Create a Conformer instance and set the initial xyz
rdkitmol.EmbedConformer()
conf = rdkitmol.GetConformer()
conf.SetPositions(xyz_dict['coords'])

Get the torsional mode and the original angles

In [ ]:
# You can set the correct (all) torsions, otherwise RDKit will perceive.
######################################
# INPUT
torsions = None
######################################
if not torsions:
    torsions = rdkitmol.GetTorsionalModes()
    print(f'RDKit perceived torsions: {torsions}')

conf.SetTorsionalModes(torsions)
num_torsions = len(torsions)
original_angles = conf.GetAllTorsionsDeg()

Example1: Sampling the angles `0, 120, 240` for each torsion <br>
For reference, a 7 heavy atom species with 5 rotors cost ~20 ms on Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz

In [ ]:
sampling = [3] * len(torsions)  # sampling = [[0., 120., 240.]] is equivalent

In [ ]:
angles_list = get_separable_angle_list(sampling,
                                       original_angles)
angle_mesh = product(*angles_list)

# Save to bookkeep
bookkeep = {}
conformers_by_change_torsions(conf,
                              angle_mesh,
                              bookkeep=bookkeep,
                              torsions=torsions,
                              on_the_fly_check=True)

Example2: Sampling the angles 45 x 45 points for each torsion pairs <br>
For reference, a 7 heavy atom species with 5 rotors cost 1.4 s on Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz

In [ ]:
n_point_each_torsion = 45
n_dimension = 2

In [ ]:
bookkeeps = []

for torsion_pair in combinations(torsions, n_dimension):
    # Reset the geometry
    conf.SetPositions(xyz_dict['coords'])
    # Get angles
    sampling = [n_point_each_torsion if tor in torsion_pair else 0
                for tor in torsions]
    angles_list = get_separable_angle_list(sampling,
                                           original_angles)
    angle_mesh = product(*angles_list)
    # Generate conformers
    bookkeep = {}
    conformers_by_change_torsions(conf,
                                  angle_mesh,
                                  bookkeep=bookkeep,
                                  torsions=torsions,
                                  on_the_fly_check=True)
    bookkeeps.append(bookkeep)

### [OPTIONAL] Check colliding atoms of conformers
if you run with on-the-fly check you can check the colliding atoms

In [ ]:
check_bookkeep = bookkeeps[0]  # Assign the dict to be checked

In [ ]:
inds = []
for ind, value in bookkeep.items():
    if value['colliding_atoms']:
        inds.append(ind)
print(f"\n{len(inds)} out of {len(bookkeep)} conformers may have colliding atoms")

### [OPTIONAL] Align Conformers
You can also align molecules stored in the `bookkeep`. The RMSD of each conformer (initial conformer as reference) is also generated `rmsd_list`

In [ ]:
to_align = bookkeeps[0]  # Assign the dict to be aligned

In [ ]:
from rdkit import Chem

rdkitmol.EmbedMultipleConfs(len(to_align))
confs = rdkitmol.GetAllConformers()
torsions_values = []
for i, value in bookkeep.items():
    confs[i].SetPositions(value['coords'])
    torsion_value = np.array(confs[i].GetAllTorsionsDeg())
    torsions_values.append(torsion_value)
rd_mol = rdkitmol.ToRDMol()

rmsd_list = []
Chem.rdMolAlign.AlignMolConformers(rd_mol, maxIters=50, RMSlist=rmsd_list)

View alignment by py3Dmol, this may take some time. {

In [ ]:
import py3Dmol

view = py3Dmol.view(width=600, height=400)
for i in range(len(bookkeep)):
    view.addModel(Chem.MolToMolBlock(rd_mol, confId=i), 'sdf')
view.setStyle({'stick':{'colorscheme':'cyanCarbon'}})
view.render()

## 2. Calculate Psi4 on the notebook

This section is only for testing instead of actual tasks.
From experience, for conformer search purpose, better to use `n_threads_each_calculation = 1` and use `n_worker` as many as possible

In [ ]:
# How many threads to use as worker
n_worker = 8  # -1 to use all threads
n_memory_each_calculation = 12000 / n_worker  # Assuming you have 
n_threads_each_calculation = 1
reference = 'uhf'
level_of_theory = 'b3lyp/def2-svp'

In [ ]:
def geom_producer(bookkeep, xyz_dict):
    for ind, conf in bookkeep.items():
        xyz_dict['coords'] = conf['coords']
        xyz_file = xyz_dict_to_xyz_file(xyz_dict)
        yield (ind, xyz_file)
        
def get_psi4_dftenergy(ind, xyz_file):
    psi4.set_memory(f'{n_memory_each_calculation} MB')
    psi4.set_options({'reference': reference})
    try:
        psi4.geometry(xyz_file)
        psi4.set_num_threads(n_threads_each_calculation)
        return (ind, psi4.energy(level_of_theory))
    except Exception as e:
        print(e)
        return (ind, 1e4)

In [ ]:
result = Parallel(n_jobs=n_worker, verbose=100) \
         (delayed(get_psi4_dftenergy)(*data) for data in geom_producer(bookkeep, xyz_dict))